## File Overview:
This script combines training and testing datasets into a single unified folder structure. It utilizes Python's os, shutil, and ThreadPoolExecutor for efficient file operations and parallel processing.

## Import Libraries
Imports required modules for file handling, directory creation, and multithreading.

In [ ]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor

## Define Directory Structure and File Copy Functions
Implements functions to create destination folder structures, copy files, and combine class samples from train and test directories.

In [ ]:
def create_directory_structure(source_path, destination_path):
    """Create destination directory structure mirroring the source."""
    for root, dirs, _ in os.walk(source_path):
        for dir_name in dirs:
            dest_dir = os.path.join(destination_path, os.path.relpath(os.path.join(root, dir_name), source_path))
            os.makedirs(dest_dir, exist_ok=True)

def copy_files(source_folder, destination_folder):
    """Copy files from source folder to destination folder."""
    for root, _, files in os.walk(source_folder):
        for file_name in files:
            source_file = os.path.join(root, file_name)
            rel_path = os.path.relpath(source_file, source_folder)
            dest_file = os.path.join(destination_folder, rel_path)
            os.makedirs(os.path.dirname(dest_file), exist_ok=True)  # Ensure directories exist
            shutil.copy2(source_file, dest_file)

def copy_class_samples(class_name, train_path, test_path, dest_path):
    """Copy files for a given class from train and test folders to destination."""
    train_class_path = os.path.join(train_path, class_name)
    test_class_path = os.path.join(test_path, class_name)
    dest_class_path = os.path.join(dest_path, class_name)

    os.makedirs(dest_class_path, exist_ok=True)

    if os.path.exists(train_class_path):
        copy_files(train_class_path, dest_class_path)
    if os.path.exists(test_class_path):
        copy_files(test_class_path, dest_class_path)

## Main Function: Dataset Integration
Combines datasets by identifying class folders in train and test directories, and processes them in parallel for efficiency.

In [1]:
def main():
    train_folder = r"C:\Users\jamee\Downloads\archive\train"  # Path to the train folder
    test_folder = r"C:\Users\jamee\Downloads\archive\test"    # Path to the test folder
    destination_folder = r"D:\Umar\Dermnet"         # Path to the destination folder

    # Ensure the destination folder exists
    os.makedirs(destination_folder, exist_ok=True)

    # Get all class folder names from train and test folders
    train_classes = os.listdir(train_folder)
    test_classes = os.listdir(test_folder)
    all_classes = set(train_classes) | set(test_classes)  # Combine unique class names

    # Parallel processing
    with ThreadPoolExecutor() as executor:
        futures = []
        for class_name in all_classes:
            futures.append(
                executor.submit(copy_class_samples, class_name, train_folder, test_folder, destination_folder)
            )
        # Ensure all tasks complete
        for future in futures:
            future.result()

if __name__ == "__main__":
    main()